<a href="https://colab.research.google.com/github/anandaditya07/Hate-Speech/blob/main/ML_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets sentencepiece scikit-learn pandas numpy emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 23.4 MB/s eta 0:00:00


1. **Import Libraries**

In [2]:
import pandas as pd
import numpy as np
import re
import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

2. **Load Dataset**

In [7]:
train_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-train.csv", encoding='latin-1')
test_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-test.csv", encoding='latin-1')

train_df.head()  #used for previewing the dataset

,tweet,label
0,à¤ à¤®à¤° à¤à¤à¤¾à¤²à¤¾ à¤¦à¥à¤¨à¤¿à¤ à¤¸à...,0
1,à¤¬à¥à¤à¥à¤ªà¥ @BJP4India à¤à¥à¤¨à¥à¤¦à...,0
2,#AssemblyElections2022 à¤ªà¥à¤°à¤­à¤¾à¤°à¥ à...,0
3,#à¤¬à¥à¤°à¥à¤à¤¿à¤à¤ - à¤®à¤§à¥à¤¯à¤ªà¥...,0
4,#AmitShah à¤¨à¥ à¤à¤¹à¤¾ à¤à¤¤à¥à¤¤à¤° à¤ª...,0


**3. Basic Data Inspection**

In [8]:
print(train_df.columns)  #Displaying all column names present in the dataset
print(train_df.isnull().sum())  #Checking missing (null) values in each column
print(train_df['label'].value_counts()) #Counting the number of samples in each class (e.g., hate vs non-hate)

Index(['tweet', 'label'], dtype='object')
tweet    0
label    0
dtype: int64
label
0    16805
1     2214
Name: count, dtype: int64


**4. Text Cleaning Function**

In [11]:
def clean_text(text):
    text = str(text).lower()                     #Converting text to lowercase for consistency.
    text = re.sub(r"http\S+|www\S+", "", text)   # Removing URLs that do not contribute to meaning.
    text = re.sub(r"@\w+|#\w+", "", text)        # Removing mentions and hashtags to reduce noise
    text = emoji.replace_emoji(text, replace="") #Removing emojis that cannot be processed by traditional NLP models.
    text = re.sub(r"[^a-zA-Z\s]", "", text)      #Removing numbers and special characters.
    text = re.sub(r"\s+", " ", text).strip()     #Removing extra spaces and formatting text properly.
    return text

**5. Apply Cleaning**

In [13]:
train_df['clean_text'] = train_df['tweet'].apply(clean_text)
test_df['clean_text'] = test_df['text'].apply(clean_text)

**6. Train–Validation Split**

In [17]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df['clean_text'],
    train_df['label'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])


Training samples: 15215
Validation samples: 3804


**7. Remove Empty Cleaned Text**

In [18]:
train_df = train_df[train_df['clean_text'].str.strip() != ""]
train_df.reset_index(drop=True, inplace=True)

print("After removing empty rows:", train_df.shape)


After removing empty rows: (3153, 3)
